In [31]:
import datetime
import numpy as np
import pandas as pd
import plotly.express as px
import warnings

import pdb

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
warnings.filterwarnings('ignore')

wbpath = 'data/DP Beh Data 2022-23.xlsm'
#read in necessary sheets 
dfcolnmes = pd.read_excel(wbpath, sheet_name='Oct', skiprows=0)
df = pd.read_excel(wbpath, sheet_name='Oct', skiprows=1)

dfinfo = pd.read_excel(wbpath, sheet_name=0)
dfstudet = pd.read_excel(wbpath, sheet_name=2, header=None)

#get years variable 
year = dfinfo.iloc[3,1]
year1, year2 = str.split(year, '-')

df.head()

,Date,Shift,No Data,Unnamed: 3,Subcategory,Unnamed: 5,Unnamed: 6,Freq,Duration Specific,Time Sampling %,Duration Intervals,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Intensity,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Comments,Subcategory.1,Unnamed: 23,Unnamed: 24,Freq.1,Duration Specific.1,Time Sampling %.1,Duration Intervals.1,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Intensity.1,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Comments.1,Subcategory.2,Unnamed: 41,Unnamed: 42,Freq.2,Duration Specific.2,Time Sampling %.2,Duration Intervals.2,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Intensity.2,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Comments.2,Subcategory.3,Unnamed: 59,Unnamed: 60,Freq.3,Duration Specific.3,Time Sampling %.3,Duration Intervals.3,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Intensity.3,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Comments.3,Subcategory.4,Unnamed: 77,Unnamed: 78,Freq.4,Duration Specific.4,Time Sampling %.4,Duration Intervals.4,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Intensity.4,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Comments.4,Unnamed: 94,Unnamed: 95,Unnamed: 96,Unnamed: 97,Unnamed: 98,Unnamed: 99,Unnamed: 100,Unnamed: 101,Unnamed: 102,Unnamed: 103,Unnamed: 104,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,7-3,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,3-11,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,11-7,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,7-3,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
dfcolnmes.head()

,October,Unnamed: 1,Unnamed: 2,Unnamed: 3,Aggression,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Self Mutilation,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,SIB,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Disruptive,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,Unnamed: 63,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Insert,Unnamed: 77,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87,Unnamed: 88,Unnamed: 89,Unnamed: 90,Unnamed: 91,Unnamed: 92,Unnamed: 93,Unnamed: 94,Bite Rel,F-Arm Catch,F-Choke Windmill,B-Choke,F-Hair Stab/Rel,B-Hair Stab/Rel,Deflection,Bite prevention,Front hair Pull Stab,Touch,Block Punch,1 Escort,2 Escort,2 Removal,1 E Seat Var,2 E Seat Var,Stand Wrap,One Arm Release,Mitts wear,Splints wear,Stuck/Combo,Frozen,Pacing
0,Date,Shift,No Data,NaN,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,Subcategory,NaN,NaN,Freq,Duration Specific,Time Sampling %,Duration Intervals,NaN,NaN,NaN,NaN,NaN,Intensity,NaN,NaN,NaN,NaN,Comments,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,<5,6-10,11-15,16-20,20+,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not Specified,0.0,0.0,0.0,0.0,0.0,Not Specified,1.0,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,7-3,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3-11,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,11-7,LOA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [33]:
#clean up column names 
df.columns = df.columns.str.replace('.1','')
df.columns = df.columns.str.replace('.2','') 
df.columns = df.columns.str.replace('.3','') 
df.columns = df.columns.str.replace('.4','') 
df.columns = df.columns.str.replace('.5','')

#create 'month number' variable 
month_name = dfcolnmes.columns[0]
datetime_object = datetime.datetime.strptime(month_name, "%B")
month_num = datetime_object.month

#if month number is between 7 and 12 year1 is selected, else year2 
if 7 <= month_num <= 12:
    year_num = year1
else:
    year_num = year2

#create date string 
date = str(year_num) + '-' + str(month_num)
print(date)

#create indexes and list variable 
idx1 = [8,26,44,62,80]
idx2 = [21,39,57,75,93]
tgt = [4,22,40,58,76]

dfdurinlist=[]
for i, enum in enumerate(idx1):
    
    dfiter = df.iloc[:,[0,1,2] + list(range(enum,idx2[i]))]
    dfiter.insert(3,'Target', str(dfcolnmes.columns[tgt[i]]))
    dfiter.drop(0, inplace=True)

    #print(pd.DataFrame(dfiter.head()))
    dfiter.columns.values[6] = 'Duration: 0'
    dfiter.columns.values[7] = 'Duration: <5'
    dfiter.columns.values[8] = 'Duration: 6-10'
    dfiter.columns.values[9] = 'Duration: 11-15'
    dfiter.columns.values[10] = 'Duration: 16-20'
    dfiter.columns.values[11] = 'Duration: 20+'
    dfiter.columns.values[12] = 'Interval: 0'
    dfiter.columns.values[13] = 'Interval: 1'
    dfiter.columns.values[14] = 'Interval: 2'
    dfiter.columns.values[15] = 'Interval: 3'
    dfiter.columns.values[16] = 'Interval: 4'

    dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 94)])
    dfiter.iloc[:,0] = dfiter.iloc[:,0].fillna(method='ffill')

    #cut off dataframe rows based on number of days in month
    if month_num == 4 or month_num == 6 or month_num == 9 or month_num == 11:
        dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 90)])
    elif month_num == 2:
        dfiter = dfiter.drop(dfiter.index[np.where(dfiter.index >= 84)])
    else:
        dfiter = dfiter
    
    dfdurinlist.append(dfiter)

dfdurin = pd.concat(dfdurinlist)
print(dfdurin.shape)
pd.DataFrame(dfdurinlist[0].head())
dfdurin.head(100)

2022-10
(465, 17)


,Date,Shift,No Data,Target,Duration Specific,Time Sampling %,Duration: 0,Duration: <5,Duration: 6-10,Duration: 11-15,Duration: 16-20,Duration: 20+,Interval: 0,Interval: 1,Interval: 2,Interval: 3,Interval: 4
1,1,7-3,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3-11,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,11-7,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,7-3,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2,3-11,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2,11-7,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,7-3,LOA,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,3,3-11,No Behaviors,Aggression,0.0,0.0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0
9,3,11-7,No Data Recorded,Aggression,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,4,7-3,NaN,Aggression,NaN,NaN,0,2,0,0,0,0,1,1.0,NaN,NaN,NaN


In [34]:
dfdurin.drop(columns=[dfdurin.columns[4],dfdurin.columns[5]], inplace=True)
durin_m = pd.melt(dfdurin, id_vars = [dfdurin.columns[0],dfdurin.columns[1],dfdurin.columns[2],dfdurin.columns[3]])
durin_m = durin_m.groupby(['Target','variable'])['value'].sum().reset_index()
durin_m = durin_m[~durin_m['Target'].str.contains('Insert')]
durin_m.head(50) 

,Target,variable,value
0,Aggression,Duration: 0,1
1,Aggression,Duration: 11-15,0
2,Aggression,Duration: 16-20,0
3,Aggression,Duration: 20+,1
4,Aggression,Duration: 6-10,16
5,Aggression,Duration: <5,25
6,Aggression,Interval: 0,3
7,Aggression,Interval: 1,13.0
8,Aggression,Interval: 2,19.0
9,Aggression,Interval: 3,1.0


In [35]:
fig = px.bar(durin_m[~durin_m['variable'].str.contains('Interval')],x = 'Target', y='value', color = 'variable',
             color_discrete_sequence= px.colors.sequential.Reds, category_orders={"variable": ["Duration: 0", "Duration: <5", "Sat", "Duration: 6-10",
                                                                                               "Duration: 11-15","Duration: 16-20", "Duration: 20+"]})
fig.update_traces(width=0.4)

fig

In [36]:
fig = px.bar(durin_m[~durin_m['variable'].str.contains('Duration')],x = 'Target', y='value', color = 'variable',
             color_discrete_sequence= px.colors.sequential.Greens, category_orders={"variable": ["Interval: 0", "Interval: 1", "Interval: 2",
                                                                                               "Interval: 3","Duration: 4"]})
fig.update_traces(width=0.4)

fig

In [37]:
fig = px.bar(durin_m[~durin_m['variable'].str.contains('Interval')],x = 'Target', y='value', color = 'variable',
             color_discrete_sequence= px.colors.sequential.Reds, category_orders={"variable": ["Duration: 0", "Duration: <5", "Sat", "Duration: 6-10",
                                                                                               "Duration: 11-15","Duration: 16-20", "Duration: 20+"]})
fig.update_traces(width=0.4)
fig

In [38]:
# dfdurin1 = df.iloc[:,[0,1,2] + list(range(8,21))]
# dfdurin1.insert(3,'Target', str(dfcolnmes.columns[4]))
# dfdurin2 = df.iloc[:,[0,1,2] + list(range(26,39))]
# dfdurin2.insert(3, 'Target', str(dfcolnmes.columns[22]))
# dfdurin3 = df.iloc[:,[0,1,2] + list(range(44,57))]
# dfdurin3.insert(3, 'Target', str(dfcolnmes.columns[40]))
# dfdurin4 = df.iloc[:,[0,1,2] + list(range(62,75))]
# dfdurin4.insert(3, 'Target', str(dfcolnmes.columns[58]))
# dfdurin5 = df.iloc[:,[0,1,2] + list(range(80,93))]
# dfdurin5.insert(3, 'Target', str(dfcolnmes.columns[76]))

# dfdurin = pd.concat([dfdurin1,dfdurin2,dfdurin3,dfdurin4,dfdurin5])              
# print(dfdurin.shape)
# dfdurin2.head()

In [39]:
#NOTE; rename 'Duration Intervals' columns with the behavior name, same as - `'Unnamed: 7':str(dfbeh.columns[3])+'_beh'`

# dfdurin = dfdurin.rename(columns = {'Unnamed: 1':'Shift','Unnamed: 2':'No Data',
#                                 'Unnamed: 11':str(dfdurin.iloc[0,7]),'Unnamed: 46':str(dfcolnmes.columns[7]),
#                                 'Unnamed: 64':str(dfcolnmes.columns[9]),'Unnamed: 79':str(dfcolnmes.columns[11])})

In [40]:
df = pd.read_excel(wbpath, sheet_name='July', skiprows=[1,2])
dfinfo = pd.read_excel(wbpath, sheet_name=0)
dfstudet = pd.read_excel(wbpath, sheet_name=2, header=None)

#get years variable 
year = dfinfo.iloc[3,1]
year1, year2 = str.split(year, '-')

#get name
# name = dfstudet.iloc[0,0]
# print(f"name is {name}")

# pdb.set_trace()
#select necessary columns and front fill NaNs in month column 
dfbeh = df.iloc[:, [0,1,2,4,7,22,25,40,43,58,61,76,79]]
#print(dfbeh.head())
dfbeh = dfbeh.rename(columns = {'Unnamed: 1':'Shift','Unnamed: 2':'No Data','Unnamed: 7':str(dfbeh.columns[3])+'_beh','Unnamed: 25':str(dfbeh.columns[5])+'_beh',
                                'Unnamed: 43':str(dfbeh.columns[7])+'_beh','Unnamed: 61':str(dfbeh.columns[9])+'_beh','Unnamed: 79':str(dfbeh.columns[11])+'_beh'})

dfbeh.head()

,July,Shift,No Data,Aggression,Aggression_beh,Self Mutilation,Self Mutilation_beh,SIB,SIB_beh,Disruptive,Disruptive_beh,Insert,Insert_beh
0,1,7-3,NaN,"hitting, grabbing",2.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN
1,NaN,3-11,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,NaN,11-7,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,2,7-3,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,NaN,3-11,No Behaviors,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


In [106]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from read_workbook import *


# Create dummy data indexed by month and with multi-columns [product, revenue]
index = pd.date_range("2020", "2021", freq="MS", closed="left")
dfdur, dfin = get_all_months_int_dur("data/DP Beh Data 2019-20.xlsm")

# dfdur_wide = pd.pivot(dfdur, columns="variable", values="value")

dfdur_wide = pd.pivot_table(dfdur, values='value', index=['Date', 'variable'], columns='Target',aggfunc=np.max)
dfdur_wide_grp = pd.pivot_table(dfdur_wide, index="Date", columns="variable", values=["Aggression", "Disruptive", "SIB", "Self Mutilation"])
dfdur_wide_grp.index.name = None

df = dfdur_wide_grp

In [107]:
# print(df.columns)
#group data by month
df['Yr_Mnth'] = pd.to_datetime(df.index).strftime('%Y-%m')
dfm = df.groupby(['Yr_Mnth'],sort=False,)[df.columns].sum().reset_index()
df = dfm.drop('Yr_Mnth', axis='columns')

In [134]:
# Create a figure with the right layout
fig = go.Figure(
    layout=go.Layout(
        height=600,
        width=1000,
        barmode="group",
        yaxis_showticklabels=False,
        yaxis_showgrid=False,
        yaxis_range=[0, df.groupby(axis=1, level=0).sum().max().max() * 1.5],
       # Secondary y-axis overlayed on the primary one and not visible
        yaxis2=go.layout.YAxis(
            visible=False,
            matches="y",
            overlaying="y",
            anchor="x",
        ),
        font=dict(size=24),
        legend_x=0,
        legend_y=1,
        legend_orientation="h",
        hovermode="x",
        margin=dict(b=0,t=10,l=0,r=10)
    )
)

# Define some colors for the product, revenue pairs
colors = {
    # "Self Mutilation" : {
    #     "Duration: 0" : "#FF0000",
    #     "Duration: 11-15": "#F00202",
    #     "Duration: 16-20": "#CA0000",
    #     "Duration: 20+": "#AF0000",
    #     "Duration: 6-10": "#900000",
    #     "Duration: <5": "#6C0000",
    # },
    "SIB" : {
        "Duration: 0" : "#00D8FF",
        "Duration: 11-15": "#00BCDE",
        "Duration: 16-20": "#00A7C6",
        "Duration: 20+": "#0089A3",
        "Duration: 6-10": "#006D82",
        "Duration: <5": "#005262",
    },
    "Aggression" : {
        "Duration: 0" : "#F7FF00",
        "Duration: 11-15": "#D6DD00",
        "Duration: 16-20": "#BAC100",
        "Duration: 20+": "#9CA200",
        "Duration: 6-10": "#838800",
        "Duration: <5": "#6A6E00",
    },
    "Disruptive" : {
        "Duration: 0" : "#00FF08",
        "Duration: 11-15": "#00ED07",
        "Duration: 16-20": "#00D006",
        "Duration: 20+": "#00AD05",
        "Duration: 6-10": "#008C04",
        "Duration: <5": "#006A03",
    },
}

# Add the traces
for i, t in enumerate(colors):
    print(i)
    base_offset = 0
    for j, col in enumerate(df[t].columns):
        # print(df[t])
        if (df[t][col] == 0).all():
            continue
        # print(df[t][col])
        # print(df.index)
        fig.add_trace(go.Bar(
            x=df.index,
            y=df[t][col],
            base=base_offset,
            # Set the right yaxis depending on the selected product (from enumerate)
            # yaxis=f"y{i + 1}",
            # Offset the bar trace, offset needs to match the width
            # The values here are in milliseconds, 1billion ms is ~1/3 month
            offsetgroup=str(i),
            #offset=(i - 1) * 1,
            #width=1,
            legendgroup=t,
            legendgrouptitle_text=t,
            name=col,
            marker_color=colors[t][col],
            marker_line=dict(width=2, color="#333"),
            hovertemplate="%{y}<extra></extra>"
                            )
                    )
        
        base_offset += df[t][col]
fig.show()
print(df.shape)

0
1
2


(12, 24)


In [113]:
dfdur_wide = pd.pivot_table(dfdur, values='value', index=['Date', 'variable'], columns='Target',aggfunc=np.max)
dfdur_wide_grp = pd.pivot_table(dfdur_wide, index="Date", columns="variable", values=["Aggression", "Disruptive", "SIB", "Self Mutilation"])
dfdur_wide_grp.index.name = None
# dfdur_wide_grp = dfdur_wide_grp.reset_index(drop=True).rename_axis(None, axis=1)
dfdur_wide_grp.head()

Target      Aggression                                                \
variable   Duration: 0 Duration: 11-15 Duration: 16-20 Duration: 20+   
2019-07-01         0.0             0.0             0.0           0.0   
2019-07-02         0.0             0.0             0.0           0.0   
2019-07-03         0.0             0.0             0.0           0.0   
2019-07-04         0.0             0.0             0.0           0.0   
2019-07-05         0.0             0.0             0.0           0.0   

Target                                  Disruptive                  \
variable   Duration: 6-10 Duration: <5 Duration: 0 Duration: 11-15   
2019-07-01            0.0          0.0         0.0             0.0   
2019-07-02            0.0          1.0         0.0             0.0   
2019-07-03            0.0          0.0         0.0             0.0   
2019-07-04            0.0          0.0         0.0             0.0   
2019-07-05            0.0          0.0         0.0             0.0   

Target                                                                \
variable   Duration: 16-20 Duration: 20+ Duration: 6-10 Duration: <5   
2019-07-01             0.0           0.0            0.0          0.0   
2019-07-02             0.0           0.0            0.0          0.0   
2019-07-03             0.0           0.0            0.0          0.0   
2019-07-04             0.0           0.0            0.0          0.0   
2019-07-05             0.0           0.0            0.0          0.0   

Target             SIB                                                \
variable   Duration: 0 Duration: 11-15 Duration: 16-20 Duration: 20+   
2019-07-01         0.0             0.0             0.0           0.0   
2019-07-02         0.0             0.0             0.0           0.0   
2019-07-03         0.0             0.0             0.0           0.0   
2019-07-04         0.0             0.0             0.0           0.0   
2019-07-05         0.0             0.0             0.0           0.0   

Target                                 Self Mutilation                  \
variable   Duration: 6-10 Duration: <5     Duration: 0 Duration: 11-15   
2019-07-01            0.0          0.0             0.0             0.0   
2019-07-02            0.0          0.0             0.0             0.0   
2019-07-03            0.0          0.0             0.0             0.0   
2019-07-04            0.0          0.0             0.0             0.0   
2019-07-05            0.0          0.0             0.0             0.0   

Target                                                                
variable   Duration: 16-20 Duration: 20+ Duration: 6-10 Duration: <5  
2019-07-01             0.0           0.0            0.0          0.0  
2019-07-02             0.0           0.0            0.0          0.0  
2019-07-03             0.0           0.0            0.0          0.0  
2019-07-04             0.0           0.0            0.0          0.0  
2019-07-05             0.0           0.0            0.0          0.0

In [51]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go


# Create dummy data indexed by month and with multi-columns [product, revenue]
index = pd.date_range("2020", "2021", freq="MS", closed="left")
df = pd.concat(
    [
        pd.DataFrame(
            np.random.rand(12, 3) * 1.25 + 0.25,
            index=index,
            columns=["Revenue1", "Revenue2", "Revenue3"]
        ),
        pd.DataFrame(
            np.random.rand(12, 3) + 0.5,
            index=index,
            columns=["Revenue1", "Revenue2", "Revenue3"]
        ),
    ],
    axis=1,
    keys=["Product1", "Product2"]
)

# Create a figure with the right layout
fig = go.Figure(
    layout=go.Layout(
        height=600,
        width=1000,
        barmode="relative",
        yaxis_showticklabels=False,
        yaxis_showgrid=False,
        yaxis_range=[0, df.groupby(axis=1, level=0).sum().max().max() * 1.5],
       # Secondary y-axis overlayed on the primary one and not visible
        yaxis2=go.layout.YAxis(
            visible=False,
            matches="y",
            overlaying="y",
            anchor="x",
        ),
        font=dict(size=24),
        legend_x=0,
        legend_y=1,
        legend_orientation="h",
        hovermode="x",
        margin=dict(b=0,t=10,l=0,r=10)
    )
)

# Define some colors for the product, revenue pairs
colors = {
    "Product1": {
        "Revenue1": "#F28F1D",
        "Revenue2": "#F6C619",
        "Revenue3": "#FADD75",
    },
    "Product2": {
        "Revenue1": "#2B6045",
        "Revenue2": "#5EB88A",
        "Revenue3": "#9ED4B9",
    }
}

# Add the traces
for i, t in enumerate(colors):
    print(t)
    for j, col in enumerate(df[t].columns):
        if (df[t][col] == 0).all():
            continue
        print(df[t][col])
        print(df.index)
        fig.add_trace(go.Bar(
            x=df.index,
            y=df[t][col],
            # Set the right yaxis depending on the selected product (from enumerate)
            yaxis=f"y{i + 1}",
            # Offset the bar trace, offset needs to match the width
            # The values here are in milliseconds, 1billion ms is ~1/3 month
            offsetgroup=str(i),
            offset=(i - 1) * 1000000000,
            width=1000000000,
            legendgroup=t,
            legendgrouptitle_text=t,
            name=col,
            marker_color=colors[t][col],
            marker_line=dict(width=2, color="#333"),
            hovertemplate="%{y}<extra></extra>"
                            )
        )

fig.show()
df.head()

Product1
2020-01-01    1.035115
2020-02-01    1.230867
2020-03-01    1.383587
2020-04-01    1.447830
2020-05-01    0.545493
2020-06-01    0.804018
2020-07-01    1.033220
2020-08-01    0.844323
2020-09-01    1.106559
2020-10-01    1.360656
2020-11-01    1.085435
2020-12-01    1.026263
Freq: MS, Name: Revenue1, dtype: float64
DatetimeIndex(['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05-01', '2020-06-01', '2020-07-01', '2020-08-01',
               '2020-09-01', '2020-10-01', '2020-11-01', '2020-12-01'],
              dtype='datetime64[ns]', freq='MS')
2020-01-01    0.443958
2020-02-01    1.312424
2020-03-01    1.251690
2020-04-01    1.256563
2020-05-01    0.929780
2020-06-01    1.261832
2020-07-01    1.436089
2020-08-01    0.677152
2020-09-01    1.121403
2020-10-01    0.262375
2020-11-01    0.540670
2020-12-01    1.264928
Freq: MS, Name: Revenue2, dtype: float64
DatetimeIndex(['2020-01-01', '2020-02-01', '2020-03-01', '2020-04-01',
               '2020-05

Product1                      Product2                    
            Revenue1  Revenue2  Revenue3  Revenue1  Revenue2  Revenue3
2020-01-01  1.035115  0.443958  0.718074  1.091501  0.506111  1.260865
2020-02-01  1.230867  1.312424  0.345752  1.493240  0.609106  1.298132
2020-03-01  1.383587  1.251690  0.441685  0.788589  1.219983  0.940186
2020-04-01  1.447830  1.256563  0.722685  1.171496  0.910978  1.396329
2020-05-01  0.545493  0.929780  0.271800  0.502419  0.952708  0.662889

In [52]:
dfdur_wide = pd.pivot_table(dfdur, values='value', index=['Date', 'variable'], columns='Target',aggfunc=np.max).reset_index()
dfdur_wide_grp = pd.pivot_table(dfdur_wide, index="Date", columns="variable", values=["Aggression", "Disruptive", "SIB", "Self Mutilation"])
dfdur_wide.head()

Target,Date,variable,Aggression,Disruptive,SIB,Self Mutilation
0,2019-07-01,Duration: 0,0.0,0.0,0.0,0.0
1,2019-07-01,Duration: 11-15,0.0,0.0,0.0,0.0
2,2019-07-01,Duration: 16-20,0.0,0.0,0.0,0.0
3,2019-07-01,Duration: 20+,0.0,0.0,0.0,0.0
4,2019-07-01,Duration: 6-10,0.0,0.0,0.0,0.0


In [53]:
dfdur_wide_grp.reset_index().head()

Target         Date  Aggression                                                \
variable            Duration: 0 Duration: 11-15 Duration: 16-20 Duration: 20+   
0        2019-07-01         0.0             0.0             0.0           0.0   
1        2019-07-02         0.0             0.0             0.0           0.0   
2        2019-07-03         0.0             0.0             0.0           0.0   
3        2019-07-04         0.0             0.0             0.0           0.0   
4        2019-07-05         0.0             0.0             0.0           0.0   

Target                                Disruptive                  \
variable Duration: 6-10 Duration: <5 Duration: 0 Duration: 11-15   
0                   0.0          0.0         0.0             0.0   
1                   0.0          1.0         0.0             0.0   
2                   0.0          0.0         0.0             0.0   
3                   0.0          0.0         0.0             0.0   
4                   0.0          0.0         0.0             0.0   

Target                                                              \
variable Duration: 16-20 Duration: 20+ Duration: 6-10 Duration: <5   
0                    0.0           0.0            0.0          0.0   
1                    0.0           0.0            0.0          0.0   
2                    0.0           0.0            0.0          0.0   
3                    0.0           0.0            0.0          0.0   
4                    0.0           0.0            0.0          0.0   

Target           SIB                                                \
variable Duration: 0 Duration: 11-15 Duration: 16-20 Duration: 20+   
0                0.0             0.0             0.0           0.0   
1                0.0             0.0             0.0           0.0   
2                0.0             0.0             0.0           0.0   
3                0.0             0.0             0.0           0.0   
4                0.0             0.0             0.0           0.0   

Target                               Self Mutilation                  \
variable Duration: 6-10 Duration: <5     Duration: 0 Duration: 11-15   
0                   0.0          0.0             0.0             0.0   
1                   0.0          0.0             0.0             0.0   
2                   0.0          0.0             0.0             0.0   
3                   0.0          0.0             0.0             0.0   
4                   0.0          0.0             0.0             0.0   

Target                                                              
variable Duration: 16-20 Duration: 20+ Duration: 6-10 Duration: <5  
0                    0.0           0.0            0.0          0.0  
1                    0.0           0.0            0.0          0.0  
2                    0.0           0.0            0.0          0.0  
3                    0.0           0.0            0.0          0.0  
4                    0.0           0.0            0.0          0.0

In [54]:
dfdur.head(10)

,Date,Target,variable,value
0,2019-07-01,Aggression,Duration: 0,0.0
1,2019-07-01,Aggression,Duration: 11-15,0.0
2,2019-07-01,Aggression,Duration: 16-20,0.0
3,2019-07-01,Aggression,Duration: 20+,0.0
4,2019-07-01,Aggression,Duration: 6-10,0.0
5,2019-07-01,Aggression,Duration: <5,0.0
11,2019-07-01,Disruptive,Duration: 0,0.0
12,2019-07-01,Disruptive,Duration: 11-15,0.0
13,2019-07-01,Disruptive,Duration: 16-20,0.0
14,2019-07-01,Disruptive,Duration: 20+,0.0
